In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm
import seaborn as sns
from scipy.stats import zscore
from scipy.stats.mstats import winsorize
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
from scipy.stats import bartlett
from scipy.stats import levene
from statsmodels.tsa.stattools import acf
from scipy.stats import jarque_bera
from scipy.stats import normaltest

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [2]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from houseprices', con=engine)

#engine.dispose()

df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


### Run your house prices model again.

In [3]:
features = ['overallqual', 'yearbuilt', 'yearremodadd', 'totalbsmtsf', 'firstflrsf', 'grlivarea', 'fullbath', 'totrmsabvgrd', 'garagecars', 'garagearea']

In [4]:
Y = df.saleprice
X = df[features]

In [5]:
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.772
Method:                 Least Squares   F-statistic:                     495.4
Date:                Thu, 31 Oct 2019   Prob (F-statistic):               0.00
Time:                        12:41:13   Log-Likelihood:                -17459.
No. Observations:                1460   AIC:                         3.494e+04
Df Residuals:                    1449   BIC:                         3.500e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.186e+06   1.29e+05     -9.187      0.000   -1.44e+06   -9.33e+05
overallqual    1.96e+04   1190.159     16.472      0.000    1.73e+04    2.19e+04
yearbuilt      268.2407     50.346      5.328      0.000     169.481     367.000
yearremodadd   296.4812     63.635      4.659      0.000     171.655     421.307
totalbsmtsf     19.8651      4.295      4.625      0.000      11.439      28.291
firstflrsf      14.1737      4.930      2.875      0.004       4.504      23.844
grlivarea       51.2971      4.233     12.119      0.000      42.994      59.600
fullbath     -6790.8715   2682.369     -2.532      0.011   -1.21e+04   -1529.130
totrmsabvgrd    33.1051   1119.061      0.030      0.976   -2162.048    2228.258
garagecars    1.042e+04   3044.272      3.422      0.001    4446.249    1.64e+04
garagearea      14.9475     10.311      1.450      0.147      -5.279      35.174
==============================================================================
Omnibus:                      477.814   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            58906.279
Skew:                          -0.412   Prob(JB):                         0.00
Kurtosis:                      34.107   Cond. No.                     4.69e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.69e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.

F-statistic is 495.4, R-squared is 0.774, adjusted R-squared is 0.772, AIC is 3.494e+04, BIC is 3.500e+04.

### Do you think your model is satisfactory? If so, why?

idk

### In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables. For each model you try, get the goodness of fit metrics and compare your models with each other. 

In [6]:
# removing half of features (least correlated)

features.remove('totrmsabvgrd')
features.remove('yearremodadd')
features.remove('yearbuilt')
features.remove('fullbath')
features.remove('totalbsmtsf')
features.remove('firstflrsf')

In [7]:
Y = df.saleprice
X = df[features]

In [8]:
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.742
Model:                            OLS   Adj. R-squared:                  0.741
Method:                 Least Squares   F-statistic:                     1045.
Date:                Thu, 31 Oct 2019   Prob (F-statistic):               0.00
Time:                        12:41:14   Log-Likelihood:                -17556.
No. Observations:                1460   AIC:                         3.512e+04
Df Residuals:                    1455   BIC:                         3.515e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -9.844e+04   4820.467    -20.420      0.000   -1.08e+05    -8.9e+04
overallqual  2.699e+04   1067.393     25.285      0.000    2.49e+04    2.91e+04
grlivarea      49.5728      2.555     19.402      0.000      44.561      54.585
garagecars   1.132e+04   3126.297      3.620      0.000    5184.992    1.75e+04
garagearea     41.4784     10.627      3.903      0.000      20.632      62.324
==============================================================================
Omnibus:                      411.179   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            12305.361
Skew:                           0.670   Prob(JB):                         0.00
Kurtosis:                      17.159   Cond. No.                     7.73e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.73e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Combining correlated features (> .80)

In [9]:
# combining correlated features (> .80)

# yearbuilt x garageyrbuilt
# grlivarea x totrmsabvgrd
# firstflrsf x totalbsmtsf
# garagearea x garagecars

# reset feature list
features = ['overallqual', 'yearbuilt', 'yearremodadd', 'totalbsmtsf', 'firstflrsf', 'grlivarea', 'fullbath', 'totrmsabvgrd', 'garagecars', 'garagearea']

#df['yearbuilt_garageyrblt'] = df.yearbuilt * df.garageyrblt
#features.append('yearbuilt_garageyrblt')
#features.remove('yearbuilt')
#features.remove('garageyrblt')

df['grlivarea_totrmsabvgrd'] = df.grlivarea * df.totrmsabvgrd
features.append('grlivarea_totrmsabvgrd')
features.remove('grlivarea')
features.remove('totrmsabvgrd')

df['firstflrsf_totalbsmtsf'] = df.firstflrsf * df.totalbsmtsf
features.append('firstflrsf_totalbsmtsf')
features.remove('firstflrsf')
features.remove('totalbsmtsf')

df['garagearea_garagecars'] = df.garagearea * df.garagecars
features.append('garagearea_garagecars')
features.remove('garagearea')
features.remove('garagecars')

In [10]:
Y = df.saleprice
X = df[features]

In [11]:
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     663.2
Date:                Thu, 31 Oct 2019   Prob (F-statistic):               0.00
Time:                        12:41:14   Log-Likelihood:                -17497.
No. Observations:                1460   AIC:                         3.501e+04
Df Residuals:                    1452   BIC:                         3.505e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                  -1.081e+06    1.3e+05     -8.286      0.000   -1.34e+06   -8.25e+05
overallqual              2.28e+04   1156.076     19.722      0.000    2.05e+04    2.51e+04
yearbuilt                283.5739     50.228      5.646      0.000     185.047     382.101
yearremodadd             252.6659     65.215      3.874      0.000     124.741     380.591
fullbath               -3981.0500   2666.647     -1.493      0.136   -9211.942    1249.842
grlivarea_totrmsabvgrd     3.7045      0.239     15.531      0.000       3.237       4.172
firstflrsf_totalbsmtsf     0.0047      0.001      4.651      0.000       0.003       0.007
garagearea_garagecars     23.0006      1.936     11.884      0.000      19.204      26.797
==============================================================================
Omnibus:                      501.155   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            56161.872
Skew:                          -0.551   Prob(JB):                         0.00
Kurtosis:                      33.364   Cond. No.                     2.36e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.36e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Which model is the best and why?

So far, my original model has been the best, since it has the highest R-squared scores.